# **El embudo de marketing: agrupación por semanas y meses**

Cuando los datos tienen ruido es difícil encontrar patrones. Por lo tanto, a veces tendrás que consolidarlo, por ejemplo, agrupándolo por semanas. Al agrupar los datos en una granularidad más pequeña (por ejemplo, de horas a semanas), las fluctuaciones se cancelan y hay menos ruido en los datos. Averigüemos cómo.

```python
import pandas as pd

df = pd.read_csv('datasets/funnel_cr_example.csv')
print(df)

#resultado
         date  clicks  regs  cr, %
0  2019-01-01    2000    36   1.80
1  2019-01-02   10000    26   0.26
2  2019-01-03    1000    15   1.50
3  2019-01-04    4000     4   0.10
4  2019-01-05    5000    24   0.48
5  2019-01-06    2000    66   3.30
6  2019-01-07    1000     2   0.20
7  2019-01-08    1500     2   0.13
```

Aquí está el primer algoritmo que viene a la mente:

1. Agrega la columna week al DataFrame.
2. Agrupa el DataFrame por esta columna.
3. Encuentra la tasa de conversión semanal media.


```python

# ¡es un error!

df['week'] = df['date'].dt.week
print(df.groupby('week')['cr, %'].mean())

week
36    1.091429
Name: cr, %, dtype: float64
```


Este enfoque es incorrecto. Bueno, los primeros dos pasos son correctos, pero el tercero no lo es. Así es como se debe hacer:

1. Agrega la columna week al DataFrame
2. Agrupa el DataFrame por esta columna.
3. Suma el número de impresiones, clics y registros de cada semana.
4. Vuelve a calcular la tasa de conversión semanal media.

Si solo promediamos los valores medios, perdemos información muy importante: el número de clics. Entonces estaríamos dando el mismo peso a la tasa de conversión del 2 de enero, cuando el sitio web tuvo 10 000 visitantes, y la tasa del 7 de enero, cuando solo hubo 1000.

```python
# correcto

df['week'] = df['date'].dt.week
df_ = df.groupby('week')[['regs', 'clicks']].sum()
print(df_['regs'] / df_['clicks'] * 100)

week
36    0.692
dtype: float64
```

## Ejercicios!

**Ejercicio 1**

Encuentra los valores semanales de CTR y CR.

Descarga los datos sobre las tasas de conversión diarias del archivo /datasets/funnel_daily.csv y guárdalos en la variable funnel_daily. Convierte los valores de la columna funnel_daily['date'] al formato datetime. Agrega la columna funnel_daily['week'] y busca allí el número de la semana para cada fecha.

Calcula el CTR y el CR semanales. Guarda el resultado como funnel_weekly.

In [ ]:
import pandas as pd

funnel_daily = pd.read_csv("/datasets/funnel_daily.csv")

funnel_daily["date"] = pd.to_datetime(funnel_daily["date"])  # escribe tu código aquí
funnel_daily["week"] = funnel_daily["date"].dt.week  # escribe tu código aquí

funnel_weekly = funnel_daily.groupby("week").agg(
    {"clicks": "sum", "impressions": "sum", "registrations": "sum"}
)
funnel_weekly["ctr, %"] = (funnel_weekly["clicks"] / funnel_weekly["impressions"]) * 100
funnel_weekly["cr, %"] = (
    funnel_weekly["registrations"] / funnel_weekly["clicks"]
) * 100

print(funnel_weekly)

"""
      clicks  impressions  registrations    ctr, %     cr, %
week
31       264        28351             23  0.931184  8.712121
32       503        57031             43  0.881976  8.548708
33       471        57184             39  0.823657  8.280255
34       424        49911             37  0.849512  8.726415
35       307        34956             25  0.878247  8.143322
"""